I would like to see the relationships between brewing methods and beer style, if any. But more importantly I want to just practice analysis in Python and writing with LaTex

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

recipes = pd.read_csv('../Data/Beer/07_beerRecipeData.csv', index_col=0, delimiter=",", encoding='latin1')

# See what we got here
recipes.head()

Name  \
BeerID                                    
1                     Vanilla Cream Ale   
2           Southern Tier Pumking clone   
3           Zombie Dust Clone - EXTRACT   
4         Zombie Dust Clone - ALL GRAIN   
5       Bakke Brygg Belgisk Blonde 50 l   

                                                      URL  \
BeerID                                                      
1            /homebrew/recipe/view/1633/vanilla-cream-ale   
2       /homebrew/recipe/view/16367/southern-tier-pumk...   
3       /homebrew/recipe/view/5920/zombie-dust-clone-e...   
4       /homebrew/recipe/view/5916/zombie-dust-clone-a...   
5       /homebrew/recipe/view/89534/bakke-brygg-belgis...   

                                     Style  StyleID  Size(L)     OG     FG  \
BeerID                                                                       
1                                Cream Ale       45    21.77  1.055  1.013   
2       Holiday/Winter Special Spiced Beer       85    20.82  1.083  1.021   
3                             American IPA        7    18.93  1.063  1.018   
4                             American IPA        7    22.71  1.061  1.017   
5                        Belgian Blond Ale       20    50.00  1.060  1.010   

         ABV    IBU  Color   ...     BoilGravity  Efficiency  MashThickness  \
BeerID                       ...                                              
1       5.48  17.65   4.83   ...           1.038        70.0            NaN   
2       8.16  60.65  15.64   ...           1.070        70.0            NaN   
3       5.91  59.25   8.98   ...             NaN        70.0            NaN   
4       5.80  54.48   8.50   ...             NaN        70.0            NaN   
5       6.48  17.84   4.57   ...           1.050        72.0            NaN   

              SugarScale  BrewMethod PitchRate PrimaryTemp  PrimingMethod  \
BeerID                                                                      
1       Specific Gravity   All Grain       NaN       17.78     corn sugar   
2       Specific Gravity   All Grain       NaN         NaN            NaN   
3       Specific Gravity     extract       NaN         NaN            NaN   
4       Specific Gravity   All Grain       NaN         NaN            NaN   
5       Specific Gravity   All Grain       NaN       19.00     Sukkerlake   

         PrimingAmount   UserId  
BeerID                           
1               4.5 oz    116.0  
2                  NaN    955.0  
3                  NaN      NaN  
4                  NaN      NaN  
5       6-7 g sukker/l  18325.0  

[5 rows x 22 columns]

**Column selection**

I think these columns will be useful, and they have data for every recipe.
* OG - In a simplified sense it's the amount of sugar before fermentation
* FG - The amount of sugar after fermenation
* ABV - The percentage of the drink which is alcohol. This is dependant on how much of the sugar was fermented. A higher original gravity and a lower final gravity means a higher ABV
* IBU - How bitter the drink is, generally related to the amount of hops
* Color - Measured in the [Lovibond scale](http://brewwiki.com/index.php/Lovibond)

Since ABV is the final result of the OG and FG it might not seem necessary to include those two, but because they are a measure of sugars, they should be able to approximate how sweet the final beer is. I'm guessing different styles will have different ranges of sweetness

In [3]:
data = recipes[['OG','FG','ABV','IBU','Color']]

**Data cleaning**

Because the data is user-generated I want to remove extreme outliers on the assumption that the user entered it wrong or used a different scale. Since their intentions are unknown I'm going to just get rid of those examples.

In [4]:
# Calculate the ABV from the OG and FG and compare it to what the user entered
# Remove recipes where the difference is large, on the assumption they could have entered other things wrong too
data = data.assign(calculatedABV=(data.OG - data.FG) * 131.25)
data = data.assign(abvDiff=np.abs(data.ABV - data.calculatedABV))
print('Removing %d beers where ABV didn\'t match calculated ABV' % data.abvDiff[data.abvDiff >= 1].count())
data = data[data.abvDiff < 0.5]

#Get rid of beer with an ABV less than 2%
print('Removing %d beers with low ABV\n' % data.ABV[data.ABV < 2].count())
data = data[data.ABV >= 2]
data = data.drop(columns=['calculatedABV', 'abvDiff'])

# The Brewer's Friend website and forums suggest that an OG of 1.07 is considered high
# See what the OG of the beer with the highest ABV is
max_abv = data.ABV.max()
max_beer = data[data.ABV == max_abv]
print('Beer with highest ABV:\n')
print(data[data.ABV == max_abv])

# See if there are any beers with an OG higher than that one's and remove them
max_og = max_beer.OG.values[0]
cnt = data.OG[data.OG > max_og].count()
print('\n%d rows with unusually high OG' % cnt)
if cnt > 0:
    data = data[data.OG <= max_og]

Removing 2001 beers where ABV didn't match calculated ABV
Removing 282 beers with low ABV

Beer with highest ABV:

         OG     FG    ABV   IBU  Color
BeerID                                
25941   1.5  1.083  54.72  3.75   50.0

0 rows with unusually high OG


**Split data**

I'm going to split the cleaned data into training, cross-validation, and test sets

In [5]:
# Shuffle the data
data = data.sample(frac=1)

# Training/CV/Test split will be 70/15/15
cnt = data.shape[0]
index1 = int(np.ceil(cnt * 0.7))
index2 = int(np.ceil(cnt * 0.85))
data_train = data.iloc[1:index1]
data_cv = data.iloc[index1+1:index2]
data_test = data.iloc[index2+1:]
print('Sizes:\nTraining set - %d\nCV set       - %d\nTest set     - %d' % (data_train.shape[0], data_cv.shape[0], data_test.shape[0]))

Sizes:
Training set - 49935
CV set       - 10700
Test set     - 10699


**Normalization**

I'm using the standard deviation to normalize each column of the training set
$$
\text{For each column }x_j\text{ calculate:}\\
μ_j = average(x_j)\\
s_j = \text{standard deviation}(x_j)\\
\text{Then recalculate the column:}\\
x_j = \frac{x_j - μ_j}{s_j}
$$
The values μ and s will then be used to normalize the other two sets

In [6]:
mu = data_train.mean()
s = data_train.std()

def normalizeData(d):
    return (d - mu) / s

train_norm = normalizeData(data_train)
cv_norm = normalizeData(data_cv)
test_norm = normalizeData(data_test)


**K-Means**

I want to see if the recipe data can be used to derive different styles of beer, so I'm going to use the K-means clustering algorithm to create clusters of recipes.
I'm creating a function to calculate the distortion because I couldn't figure out how to do it with sci-kit.
$$
\text{The distortion }J\text{ is measured as the average squared distance between each data point and the cluster center it has been assigned to.}\\
x = \text{The data set}\\
μ = \text{The cluster centers}\\
c = \text{A vector indicating which cluster center each row of the data set is assigned to}\\
m = \text{The number of rows in the data set}\\
J = \frac{1}{m}\sum_{m}^{i=1}\lVert x^{(i)} - μ_{c^{(i)}} \rVert ^2
$$
I'm running the algorithm multiple times with different numbers of clusters so I can then decide a good amount. The original data grouped the recipes into 176 different styles so I'm trying between 10 and 180

In [ ]:
from sklearn.cluster import KMeans

# Calculate distortion the old fashioned way, unoptimized
def calculate_distortion(model, X):
    centroids = model.cluster_centers_
    indices = model.predict(X)
    m = X.shape[0]
    
    J = 0
    for i in range(0,m):
        c = centroids[indices[i]]
        J += (X.iloc[i] - c).sum() ** 2
    return J/m

# There are 176 unique styles in the original data
cluster_counts = np.arange(10,180,10)
models = [None] * len(cluster_counts)
costs = [None] * len(cluster_counts)

# This is a long process because I don't know much about optimization
for i in np.arange(0,len(cluster_counts)):
    models[i] = KMeans(n_clusters = cluster_counts[i]).fit(train_norm)
    costs[i] = calculate_distortion(models[i], cv_norm)

One way to figure out a good number of clusters is to graph the distortion for each choice of cluster count.

If we are lucky there will be a noticable point where adding more clusters does not decrease the distortion very much.

This is how the "elbow" method gets its name

In [7]:
#Graph the data
plt.figure(figsize=(10,6))
plt.plot(cluster_counts, costs, 'o')
plt.xlabel('Clusters ( K )')
plt.ylabel('Distortion ( J )')
plt.show()

The graph is pretty curved but somewhere between 40 and 100 could be good

In [79]:
# 80 is at index 7
selected_count = 7
K = cluster_counts[selected_count]
model = models[selected_count]

# Now we can see the distortion on the test set
test_distortion = calculate_distortion(model, test_norm)
print('Distortion on test set with %d clusters: %f' % (K, test_distortion))

Now I can append the new "styles" to the original data and see how it compares to what the users entered

In [80]:
def append_style(m, X):
    indices = m.predict(X)
    return X.assign(clusterStyle = indices)

train_style = append_style(model, train_norm)
cv_style = append_style(model, cv_norm)
test_style = append_style(model, test_norm)

style_data = train_style.append(cv_style).append(test_style).sort_index()['clusterStyle']
full_data = recipes.join(style_data)

See how many recipes fall into each of the new styles

In [81]:
counts = full_data.sort_values(['clusterStyle'])['clusterStyle'].value_counts().sort_index()
plt.figure(figsize=(10,6))
plt.xlabel('Class')
plt.ylabel('# Members')
counts.plot.bar()

A heatmap should give a good visualization of how the new classes relate to the ones the users assigned.

This first block creates the data by adding how how many of each original style belong to each new style.

Then it divides that by the total amount of each original style

In [82]:
full_data = full_data[np.isfinite(full_data['clusterStyle'])]
styleMap = np.zeros((K, 176))

for index, row in full_data.iterrows():
    styleMap[int(row.clusterStyle)-1][row.StyleID-1] += 1

# Get the total amount of each original StyleID
totals = np.zeros(176)

for i in range(len(totals)):
    totals[i] = full_data.StyleID[full_data.StyleID==(i+1)].count()

# Divide the counts in the style map by the total for each StyleID
for i in range(styleMap.shape[0]):
    for j in range(styleMap.shape[1]):
        styleMap[i][j] /= totals[j]

In [84]:
plt.figure(figsize=(20,20))
heatmap = plt.pcolor(styleMap, cmap=plt.cm.binary)
plt.yticks(np.arange(0, K, 1))
plt.xticks(np.arange(0, 176, 10))
plt.colorbar(heatmap)
plt.xlabel('Original StyleID')
plt.ylabel('New Class')
plt.show()

Darker rectangles mean a specific original style is grouped mostly in that new class. Long horizontal bands mean a new class encompasses a wide range of original styles. It is my intuition that if the data was able to be predictive of class we would want as few long horizontal bands as possible

Conclustion:

It's somewhat difficult for me to draw conclusions from this. Perhaps including the name of the original styles would help but because of the sheer number of them I don't know how to visualize that without cluttering the graph.

More data about the brewing process may be needed, especially because the features I used were somewhat correlated with each other. Some of the features I didn't use (Like mash thickness?) could be useful but they don't have data for every row and I don't know how to handle incomplete data yet. I would also like to scrape more data in the future such as the type of yeast used in the recipe.